In [1]:
import os 
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0,2"

In [2]:
import torch
import random
import numpy as np

from transformers import set_seed
# Set seed for random number generators in Python, NumPy, and PyTorch
seed = 32
set_seed(32)
# # 1. Set the seed for Python's built-in random module
# random.seed(seed)

# # 2. Set the seed for NumPy (if you're using it)
# np.random.seed(seed)

# # 3. Set the seed for PyTorch
# torch.manual_seed(seed)

# # 4. If you are using CUDA
# if torch.cuda.is_available():
#     torch.cuda.manual_seed(seed)
#     torch.cuda.manual_seed_all(seed)  # If using multi-GPU

# # 5. Make sure that operations are deterministic (this can slow down training, though)
# torch.backends.cudnn.deterministic = True
# torch.backends.cudnn.benchmark = False


In [3]:
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM,AutoTokenizer

In [4]:
model_id = "meta-llama/Llama-2-13b-chat-hf"
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

In [5]:
config = PeftConfig.from_pretrained("zera09/llama_FT")
base_model = AutoModelForCausalLM.from_pretrained(model_id,load_in_4bit=True, device_map='cuda')
model = PeftModel.from_pretrained(base_model, "zera09/llama_FT")


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

RuntimeError: r.nvmlDeviceGetNvLinkRemoteDeviceType_ INTERNAL ASSERT FAILED at "../c10/cuda/driver_api.cpp":27, please report a bug to PyTorch. Can't find nvmlDeviceGetNvLinkRemoteDeviceType: /lib/x86_64-linux-gnu/libnvidia-ml.so.1: undefined symbol: nvmlDeviceGetNvLinkRemoteDeviceType

In [ ]:
template = """### Instruction
Given this context: Given this current price:{price} with the prices for 10 days: {prices} and the trend: {cpi} and {news} output only one decision for the stock from the square bracket [buy,sell,hold].
### Response:
Decision:```"""


In [ ]:
import pandas as pd

data = pd.read_pickle('MSFT_train.pkl')
data = pd.DataFrame(data).T

In [ ]:
def format_data(data):
    prices= []
    news = []
    for idx,row in data.iterrows():
        prices.append(row['price']['DRIV'])
        news.append(row['news']['DRIV'][0] if row['news']['DRIV'] else 'No News' )
    return {'prices':prices,
           'news':news}
ans = format_data(data)

In [ ]:
def window_input(prices,cur_p):
    if len(prices)==0:
        return []
    p1= float(prices[0])
    
    PRICES = prices
    cpi = []
    for p in PRICES:
        # if float(p)>p1:
        #     cpi.append((p,'Price Increase'))
        # elif float(p)==p1:
        #     cpi.append((p,'Price Same'))
        # else:
        #     cpi.append((p,'Price Decrease'))
        if float(p)>p1:
            cpi.append('Price Increase for '+str(p))
        elif float(p)==p1:
            cpi.append('Price Same for '+str(p))
        else:
            cpi.append('Price Decrease for '+str(p))
            
    return cpi
# window_input(ans['prices'][:10],ans['prices'][10])

In [ ]:
def get_input(data, k=10):
    result = []
    prices = data['prices']
    news = data['news']
    
    model.eval()
    ans_list = []
    for i in range(len(prices)):
        start = max(0, i - k)
        previous_prices = prices[start:i]
        previous_news = news[start:i]
        cpi = window_input(previous_prices,prices[i])
        
        prompt = template.format(news=previous_news,price=prices[i],prices= previous_prices ,cpi=cpi)
        # print(i,prompt)
        #print(previous_prices)
        #break
        toks = tokenizer(prompt, return_tensors="pt",max_length=4096,truncation=True,padding=True).to("cuda")
        with torch.no_grad():
            out = model.generate(
                **toks,
                max_new_tokens=10,
                top_k=5,
                do_sample=True,
            )
        ans = tokenizer.decode(out[0][len(toks["input_ids"][0]):], skip_special_tokens=True)
        print(i,ans.split('```')[0])
        ans_list.append(ans)
        
    return ans_list
                        
                


output = get_input(ans)

In [ ]:
answer = []
for i in output:
    a = i.split('``')[0]
    a=a.split('\n')
    lower_list = [item.lower() for item in a]
    if 'buy' in lower_list:
        answer.append(1)
    elif 'hold' in lower_list:
        answer.append(0)
    elif 'sell' in lower_list:
        answer.append(-1)
    else:
        answer.append(0)


In [ ]:
import numpy as np
import pandas as pd



# Get actions for different models
def get_action(start, end, ticker, file_path, col):
    """
    Fetch action data from a specified CSV file.

    Parameters:
    start (str): Start date for the action data.
    end (str): End date for the action data.
    ticker (str): Ticker symbol of the stock (not used in the function).
    file_path (str): Path to the CSV file containing the action data.
    col (list): Column names to be used for date and action.

    Returns:
    list: List of actions corresponding to the specified date range.
    """
    df = pd.read_csv(file_path)
    df[col[0]] = pd.to_datetime(df[col[0]])
    action = df[(df[col[0]] >= pd.to_datetime(start)) & (df[col[0]] < pd.to_datetime(end))][col[1]].tolist()
    print(action)
    return action

def daily_reward(price, actions):
    """
    Calculate daily rewards based on price changes and actions.

    Parameters:
    price (list): List of prices.
    actions (list): List of actions taken.

    Returns:
    list: Daily rewards.
    """
    reward = []

    for i in range(len(price) - 1):
        r = actions[i] * np.log(price[i+1] / price[i])
        reward.append(r)
    return reward

def standard_deviation(reward):
    """
    Calculate the daily standard deviation of rewards.

    Parameters:
    reward (list): List of rewards.

    Returns:
    float: Standard deviation of the rewards.
    """
    mean = sum(reward) / len(reward)
    variance = sum((r - mean) ** 2 for r in reward) / (len(reward) - 1)
    return variance ** 0.5

def total_reward(price, actions):
    """
    Calculate the cumulative return over the trading period.

    Parameters:
    price (list): List of prices.
    actions (list): List of actions taken.

    Returns:
    float: Total cumulative reward.
    """
    reward = 0
    for i in range(len(price) - 1):
        reward += actions[i] * np.log(price[i+1] / price[i])
    return reward

def annualized_volatility(daily_std_dev, trading_days=252):
    """
    Calculate the annualized volatility from daily standard deviation.

    Parameters:
    daily_std_dev (float): Standard deviation of daily returns.
    trading_days (int): Number of trading days in a year, typically 252.

    Returns:
    float: Annualized volatility.
    """
    return daily_std_dev * (trading_days ** 0.5)

def calculate_sharpe_ratio(Rp, Rf, sigma_p, price):
    """
    Calculate the Sharpe ratio of an investment.

    Parameters:
    Rp (float): Return of the portfolio.
    Rf (float): Risk-free rate (annualized).
    sigma_p (float): Standard deviation of the portfolio's excess return.

    Returns:
    float: Sharpe ratio.
    """
    if sigma_p == 0:
        raise ValueError("Standard deviation cannot be zero.")
    Rp = Rp / (len(price)/252)

    return (Rp - Rf) / sigma_p

def calculate_max_drawdown(daily_returns):
    """
    Calculate the maximum drawdown of a portfolio.

    Parameters:
    daily_returns (list): List of daily returns.

    Returns:
    float: Maximum drawdown.
    """
    cumulative_returns = [1]
    for r in daily_returns:
        cumulative_returns.append(cumulative_returns[-1] * (1 + r))

    peak = cumulative_returns[0]
    max_drawdown = 0

    for r in cumulative_returns:
        if r > peak:
            peak = r
        drawdown = (peak - r) / peak
        if drawdown > max_drawdown:
            max_drawdown = drawdown

    return max_drawdown

def calculate_metrics(price, actions):
    """
    Calculate various financial metrics based on price and actions.

    Parameters:
    price (list): List of daily prices.
    actions (list): List of actions taken.

    Returns:
    tuple: A tuple containing calculated metrics (standard deviation, annualized volatility, cumulative return, Sharpe ratio, max drawdown).
    """
    daily_rw = daily_reward(price, actions)
    std_dev_r = standard_deviation(daily_rw)


    ann_vol = annualized_volatility(std_dev_r)
    cum_return = total_reward(price, actions)
    sharpe_ratio = calculate_sharpe_ratio(cum_return, 0, ann_vol, price)
    max_dd = calculate_max_drawdown(daily_rw)
    return cum_return, sharpe_ratio, std_dev_r, ann_vol, max_dd



In [ ]:
#metrics = ['Cumulative Return', 'Sharpe Ratio', 'Standard Deviation', 'Annualized Volatility', 'Max Drawdown']
calculate_metrics(ans['prices'], answer)
